In [1]:
!pip freeze | grep scikit-learn

STDIN
scikit-learn==1.0.2


In [9]:
import pickle
import pandas as pd
import numpy as np

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [10]:
np.mean(y_pred)

16.191691679979066

In [11]:
year=2021
month=2

In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [23]:
df_result = pd.concat([df['ride_id'].reset_index(drop=True), pd.DataFrame(y_pred, columns=['pred'])], axis=1)

In [24]:
output_file = 'df_result'

In [25]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [ ]:
jupyter nbconvert --to script starter.ipynb

In [ ]:
"sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"